
**Controller functions, what are they?**

These are functions that control (read/write) into the json file in order to automate the process. You don't need to understand the UI to write them.

Here are some examples:


In [1]:

import json

def read_json():
    with open('db.json', 'r') as f:
        return json.load(f)
    

def user_exists(email, data):
    """Checks if a user with a specific email exists in the database."""
    for user in data['PomodorosApp']['Users']:
        if user['Email'] == email:
            return True
    return False


# should return True because user is in the json
print ( f"Does user exist?:", user_exists("ramy@ramy.nl", read_json()) )


# print a list of all user's projects:
def get_user_projects(email, data):
    """Returns a list of a user's projects."""
    for user in data['PomodorosApp']['Users']:
        if user['Email'] == email:
            return [project['ProjectName'] for project in user['Projects']]
    return []


print ("User's projects are: " , get_user_projects("ramy@ramy.nl", read_json())  )





def save_json(data):
    with open("db.json", 'w') as f:
        json.dump(data, f)




def add_project(user_email, project_name):

    data = read_json()

    user = next((u for u in data["PomodorosApp"]["Users"] if u["Email"] == user_email), None)

    if user is None:
        return (f"No user found with email {user_email}")
    elif any(p["ProjectName"] == project_name for p in user["Projects"]):
        return (f"Project {project_name} already exists.")
    else:
        # Create a new project
        new_project = {
            "ProjectName": project_name,
            "ProjectTotalTrackedTime": "0",
            "Subjects": []
        }
        user["Projects"].append(new_project)
        save_json(data)
        return True
    

print ("Adding project to user (should return true if added): ", add_project("ramy@ramy.nl", "sdfsdf"))





def create_user(email, name):
    data = read_json()
    new_user = {
        "Email": email,
        "Name": name,
        "Recipients": [email], # in the project's PDF, it says the the user's email should be included by default in the Recipients list
        "Projects": []
    }
    data["PomodorosApp"]["Users"].append(new_user)
    save_json(data)


# adding new user. No need for error check because this can be done by user_exists()

newUserName = "Mr. Cool"
newUserEmail = "whatever@cool.com" # try with a different email
data = read_json()

if not user_exists(newUserEmail, data):
    create_user(newUserEmail, newUserName)
    print ("User created.")
else:
    print("user already exist!")





# show subjects under a specifc project

def get_project_subjects(email, project_name, data):
    """Returns a list of subjects under a specific project for a user."""
    for user in data['PomodorosApp']['Users']:
        if user['Email'] == email:
            for project in user['Projects']:
                if project['ProjectName'] == project_name:
                    return [subject['SubjectName'] for subject in project['Subjects']]
    return []


listofSubjects = get_project_subjects('ramy@ramy.nl', 'Project01', data)

print ("List of subjects under a specific project", listofSubjects)

# becaue it's a list, it can be printed easily list so:

for subject in listofSubjects:
    print (subject)








def add_subject(user_email, project_name, subject_name):
    data = read_json()
    user = next((u for u in data["PomodorosApp"]["Users"] if u["Email"] == user_email), None)
    project = next((p for p in user["Projects"] if p["ProjectName"] == project_name), None)

    if user is None:
        return (f"No user found with email {user_email}")
    elif project is None:
        return (f"No project found with name {project_name}")
    else:

        new_subject = {
            "SubjectName": subject_name,
            "SubjectTotalTrackedTime": "0",
            "PomodoroSessions": []
        }
        project["Subjects"].append(new_subject)
        save_json(data)
        return True

newSubject = "sooobjeeeekttt"
add = add_subject ('ramy@ramy.nl', 'Project01', newSubject)

print ("adding subject under project:", add)







# Here's something a bit complex:
# its a loop inside of a loop inside of a loop, etc.

def get_session_details(email, project_name, subject_name, data):
    """Returns details about each PomodoroSession for a specific subject."""
    for user in data['PomodorosApp']['Users']:  # Like Bahdir said, this is not necessary but visually it helps understand what i am looping 
        if user['Email'] == email: # find the user by their email 
            for project in user['Projects']: # loop through the projects
                if project['ProjectName'] == project_name: #if the project name mathces what you're looking for then
                    for subject in project['Subjects']: # loop through the subjects
                        if subject['SubjectName'] == subject_name: #if name match
                            session_details = [] #create a list
                            for session in subject['PomodoroSessions']: # loop through the PomodoroSessions
                                completed_tasks = [task['TaskName'] for task in session['Tasks'] if task['Completed'] == 'True'] # find completed tasks and add them to the list
                                uncompleted_tasks = [task['TaskName'] for task in session['Tasks'] if task['Completed'] != 'True'] # same but for not completed tasks
                                session_details.append({ # adds a dict
                                    'StartTimestamp': session['StartTimestamp'],
                                    'EndTimestamp': session['EndTimestamp'],
                                    'CompletedTasks': completed_tasks,
                                    'UncompletedTasks': uncompleted_tasks
                                })
                            return session_details # returns session_details
    return [] # returns an empty list if error


data = read_json()

session_details = get_session_details('ramy@ramy.nl', 'Project01', 'Subject01', data)
print("Session details:")
for session in session_details:
    print(f"- StartTimestamp: {session['StartTimestamp']}")
    print(f"  EndTimestamp: {session['EndTimestamp']}")
    print(f"  CompletedTasks: {session['CompletedTasks']}")
    print(f"  UncompletedTasks: {session['UncompletedTasks']}")


Does user exist?: True
User's projects are:  ['Project01', 'proooojeect', 'sdfsdf']
Adding project to user (should return true if added):  Project sdfsdf already exists.
user already exist!
List of subjects under a specific project ['Subject01', 'sooobjeeeekttt', 'sooobjeeeekttt', 'sooobjeeeekttt', 'sooobjeeeekttt', 'sooobjeeeekttt', 'sooobjeeeekttt', 'sooobjeeeekttt', 'sooobjeeeekttt']
Subject01
sooobjeeeekttt
sooobjeeeekttt
sooobjeeeekttt
sooobjeeeekttt
sooobjeeeekttt
sooobjeeeekttt
sooobjeeeekttt
sooobjeeeekttt
adding subject under project: True
Session details:
- StartTimestamp: 1677492925.125641
  EndTimestamp: 1677493119.798744
  CompletedTasks: ['Task01', 'Task03']
  UncompletedTasks: ['Task02']
- StartTimestamp: 1677492925.125641
  EndTimestamp: 1677493028.564154
  CompletedTasks: ['Task01', 'Task03']
  UncompletedTasks: ['Task02']


In [ ]:
#Checks if task compated. 
def is_task_completed(email, project_name, subject_name, task_name, data) :
    for dict_ in data["PomodorosApp"]["Users"]:
       if dict_["Email"]==email:
           for dict_2 in dict_["Projects"]:
               if dict_2["ProjectName"]==project_name:
                   for dict_3 in dict_2["Subjects"]:
                       if dict_3["SubjectName"]==subject_name:
                           for dict_4 in dict_3["PomodoroSessions"]:
                              for dict_5 in dict_4["Tasks"]:
                                  if dict_5["TaskName"]==task_name:
                                      return dict_5["Completed"]
                           raise Exception("Task not found")
                   



#Test
# print(is_task_completed("ramy@ramy.nl","Project01"  , "Subject01", "Task02", data))


    


Hopefully you got the idea of what a controller function is. What I wrote above is not perfect, but it can help you get the idea. add_subject() function for example is missing error check if the subject already exisit

# Functions we need:

  

 - is_task_completed(email, project_name, subject_name, task_name, data) # should return True or False
 - get_project_total_time(email, project_name, data) # should returns
   the total time on the project result
 - get_user_recipients(email, data) # should return a list with the
   emails
 -  get_subject_task_names(email, project_name, subject_name, data) #
   should return the task names for each task under a specific subject.
 - get_username_from_email (email, data) # you have a user's email
   address, return their name from the json
 
 
 - add_task(user_email, project_name, subject_name, task_name,
   completed) # adding a task. The var "completed" should be the string
   "True" or "False" and save the task in the json

  
  
## Also:
as much as possible, try to implement error checks before saving data to the json (for example does this task/project/subject/whatever already exist? )

We might need more controller functions as we go on, for example controllers to format data (e.g. turning a unix timestamp to a normal date with day and month and year).

**When you finish what you're working on, just send a pull request to me on GitHub!**


# Code error? put it on Github!!!! Questions? send them on slack!




